In [1]:
# -*- coding: utf-8 -*-
"""
Macro & Sentiment position back‑test
===================================
Inputs
------
1. 宏观体系打分.xlsx     (周频)  必含列: Date, Quantile
2. 情绪指标回测数据.xlsx (日频)  必含列: Date, 总得分, 上证指数T+1收益率(%)

Outputs
-------
merged_positions.xlsx            # 合并后带仓位/收益
perf_summary.xlsx                # 绩效总览 + 净值
excess_net_value_all.png         # 三策略超额净值曲线
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ========== 配置 ==========
CONFIG = {
    'MACRO_FILE':   '宏观体系打分.xlsx',
    'SENT_FILE':    '情绪指标回测数据.xlsx',
    'OUT_DIR':      '',      # 所有输出都会放到这里
}
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置微软雅黑/黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示异常
# ==========================

In [3]:
# ---------- 1. 读取 & 生成信号 ---------------------------------------------
macro = pd.read_excel(CONFIG['MACRO_FILE'], parse_dates=['Date'])

def quantile_to_signal(q):
    if q >= 0.90:
        return 2
    elif q >= 0.69:
        return 1
    elif q >= 0.365:
        return 0
    elif q >= 0.20:
        return -1
    else:
        return -2

macro['Signal_macro'] = macro['宏观打分历史分位数'].apply(quantile_to_signal)
# # 2.2 2‑day same‑signal confirmation
# confirmed = []
# prev_pos = 0
# for i in range(len(macro)):
#     if i == 0:
#         confirmed.append(prev_pos)
#         continue
#     if macro.loc[i,'Signal_macro_raw'] == macro.loc[i-1,'Signal_macro_raw'] and \
#        macro.loc[i,'Signal_macro_raw'] != prev_pos:
#         prev_pos = macro.loc[i,'Signal_macro_raw']
#     confirmed.append(prev_pos)
# macro['Signal_macro'] = confirmed


sent = pd.read_excel(CONFIG['SENT_FILE'], parse_dates=['Date'])
sent = sent.sort_values('Date')

# 2‑sigma 阈值
M, SD = 0.24, 1.15

# 2.1 raw signal
raw = []
prev = 0
for sc in sent['总得分']:
    if sc >= M + 2*SD:
        sig = 2
    elif sc >= M + SD:
        sig = 2 if prev == 2 else 1
    elif sc >= M:
        sig = prev if prev in (2,1) else 0
    elif sc <= M - 2*SD:
        sig = -2
    elif sc <= M - SD:
        sig = -2 if prev == -2 else -1
    else:
        sig = prev if prev in (-2,-1) else 0
    raw.append(sig)
    prev = sig
sent['RawSignal'] = raw

# 2.2 2‑day same‑signal confirmation
confirmed = []
prev_pos = 0
for i in range(len(sent)):
    if i == 0:
        confirmed.append(prev_pos)
        continue
    if sent.loc[i,'RawSignal'] == sent.loc[i-1,'RawSignal'] and \
       sent.loc[i,'RawSignal'] != prev_pos:
        prev_pos = sent.loc[i,'RawSignal']
    confirmed.append(prev_pos)
sent['Signal_sentiment'] = confirmed

sent = sent[['Date','Signal_sentiment','上证指数T+1收益率', '国债指数T+1收益率']]

In [4]:
# ---------- 2. 合并 & 向后填充宏观仓位 --------------------------------------
df = pd.merge_asof(sent.sort_values('Date'),
                   macro.sort_values('Date'),
                   on='Date',
                   direction='backward')

In [5]:
# 信号 → 仓位映射 (同情绪规则)
sig2pos = {2:0.8, 1:0.65, 0:0.5, -1:0.35, -2:0.2}
df['Position_macro'] = df['Signal_macro'].map(sig2pos)
df['Position_sentiment'] = df['Signal_sentiment'].map(sig2pos)

In [6]:
# Create new combination strategy
# Keep macro position unchanged, adjust sentiment by adding/subtracting percentages
sentiment_adj = {2: 0.10, 1: 0.05, 0: 0.00, -1: -0.05, -2: -0.10}
# 宏观主导，情绪辅助
df['Position_sentiment_adj'] = df['Position_macro'] + df['Signal_sentiment'].map(sentiment_adj)
# 情绪主导，宏观辅助
df['Position_macro_adj'] = df['Position_sentiment'] + df['Signal_macro'].map(sentiment_adj)
# # Ensure positions are within [0.2, 0.8] range
# df['Position_sentiment_adj'] = df['Position_sentiment_adj'].clip(0.2, 0.8)


df['combo_opt_pos'] = df['Position_sentiment_adj']
mask = (df['Signal_sentiment'] == 2) & (df['Signal_macro'] <= -1)
df.loc[mask, 'combo_opt_pos'] = 0.8
mask2 = (df['Signal_sentiment'] == -2) & (df['Signal_macro'] >= 1)
df.loc[mask2, 'combo_opt_pos'] = 0.5

In [7]:
# 筛选条件：情绪信号为0且宏观信号为-1
mask = (df['Signal_sentiment'] == 0) & (df['Signal_macro'] == -1)
filtered_df = df[mask]

In [9]:
# ---------- 3. 回测 ---------------------------------------------------------
df = df.dropna().reset_index(drop=True)
df['ret']       = df['上证指数T+1收益率']/100.0
df['bond_ret']  = df['国债指数T+1收益率']/100.0  # 新增国债收益

# 修改基准：仓位50%股票+50%国债
bench = 0.5 * df['ret'] + 0.5 * df['bond_ret']

# 策略收益计算：剩余仓位投资国债
sent_ret    = df['Position_sentiment'] * df['ret'] + (1-df['Position_sentiment']) * df['bond_ret']
macro_ret   = df['Position_macro'] * df['ret'] + (1-df['Position_macro']) * df['bond_ret']
combo_ret   = 0.5*sent_ret + 0.5*macro_ret   # 50/50 组合

# 新策略收益计算
combo_adj_ret = df['Position_sentiment_adj'] * df['ret'] + (1-df['Position_sentiment_adj']) * df['bond_ret']

combo_adj_ret2 = df['Position_macro_adj'] * df['ret'] + (1-df['Position_macro_adj']) * df['bond_ret']

# 优化策略收益计算
combo_opt_ret = df['combo_opt_pos'] * df['ret'] + (1-df['combo_opt_pos']) * df['bond_ret']
combo_opt_ex  = combo_opt_ret - bench

In [10]:
# ---------- 4. 绩效函数 ------------------------------------------------------
def perf(r, rf=0.02):
    rf_d = rf/252
    cum = (1+r).cumprod()
    total = cum.iat[-1]-1
    yrs = len(r)/252
    ann = (1+total)**(1/yrs)-1
    vol = r.std()*np.sqrt(252)
    sharpe = (r.mean()-rf_d)*np.sqrt(252)/r.std()
    mdd = (cum/cum.cummax()-1).min()
    return total, ann, vol, sharpe, mdd

strategies = {
        '情绪指标': sent_ret, 
          '宏观指标': macro_ret,
          '平均加权': combo_ret,
          '宏观主导+情绪辅助' : combo_adj_ret,
          '宏观主导+情绪辅助+情绪强烈看多优化' : combo_opt_ret,
          '情绪主导+宏观辅助': combo_adj_ret2,
          '基准' : bench
}

rows = []
for name, r in strategies.items():
    rows.append((name, *perf(r)))


perf_df = pd.DataFrame(rows, columns=[
    '策略','总收益','年化收益','年化波动率','夏普比率','最大回撤']).round(4)

# 格式化显示：将除信息比率外的指标转为百分比
for col in ['总收益','年化收益','年化波动率','最大回撤']:
    perf_df[col] = perf_df[col].apply(lambda x: f'{x*100:.2f}%')

# 信息比率保留3位小数
perf_df['夏普比率'] = perf_df['夏普比率'].round(3)

# out_perf = os.path.join(CONFIG['OUT_DIR'],'perf_summary.xlsx')
# with pd.ExcelWriter(out_perf) as w:
#     perf_df.to_excel(w, sheet_name='策略', index=False)
display(perf_df)

,策略,总收益,年化收益,年化波动率,夏普比率,最大回撤
0,情绪指标,47.18%,7.88%,8.33%,0.712,-11.64%
1,宏观指标,52.28%,8.60%,8.13%,0.810,-7.90%
2,平均加权,49.94%,8.27%,7.86%,0.796,-8.06%
3,宏观主导+情绪辅助,59.89%,9.65%,8.00%,0.942,-7.61%
4,宏观主导+情绪辅助+情绪强烈看多优化,69.57%,10.92%,8.10%,1.073,-7.61%
5,情绪主导+宏观辅助,56.30%,9.16%,8.12%,0.874,-9.39%
6,基准,27.53%,4.89%,8.06%,0.384,-10.67%


In [ ]:
plt.figure(figsize=(12,6))
for name, r in strategies.items():
    plt.plot(df['Date'], (1 + r).cumprod(), label=name, linewidth=1.6)
# plt.title("Net Value Curves – Strategies vs Benchmark")
plt.ylabel("净值")
plt.axhline(1.0, color="grey", lw=1, ls="--")
plt.legend(); plt.grid(alpha=0.3); plt.tight_layout()
plt.show()

In [ ]:
# 计算相对超额收益绩效
excess_rel_perf = []
for name, s in strategies.items():
    if name != '基准':
        # 算术超额
        rel_excess = s - bench
        # 计算绩效指标
        excess_rel_perf.append((name, *perf(rel_excess, rf=0)))

rel_perf_df = pd.DataFrame(excess_rel_perf, columns=[
    '策略','相对超额总收益','相对超额年化收益','相对超额波动率','夏普比率','最大相对回撤'])

# 格式化显示
for col in ['相对超额总收益','相对超额年化收益','相对超额波动率','最大相对回撤']:
    rel_perf_df[col] = rel_perf_df[col].apply(lambda x: f'{x*100:.2f}%')

rel_perf_df['夏普比率'] = rel_perf_df['夏普比率'].round(3)

display(rel_perf_df)

In [ ]:
# 计算相对比率的超额收益情况
excess_rel_nv = pd.DataFrame({'Date':df['Date']})

# 计算相对比率的超额收益
for k,s in strategies.items():
    if k != '基准':  # 排除基准曲线
        # 相对比率法计算超额收益:策略收益/基准收益-1 
        rel_excess = (1 + s)/(1 + bench) - 1
        excess_rel_nv[k] = (1 + rel_excess).cumprod()

# 绘制相对超额收益曲线        
plt.figure(figsize=(12,6))
for k in excess_rel_nv.columns:
    if k != 'Date':
        plt.plot(excess_rel_nv['Date'], excess_rel_nv[k], label=k, linewidth=1.6)

plt.axhline(1.0, ls='--', lw=1, color='grey', alpha=0.5)
plt.title('策略相对超额收益净值曲线')  
plt.xlabel('日期')
plt.ylabel('相对超额净值')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# df['combo_opt_pos'] = df['Position_sentiment_adj']
combo_adj_pos = df['combo_opt_pos']       # 保证 0~1 之间

# 2. 策略日收益 & 超额收益
excess_nv_combo = (1 + combo_opt_ex).cumprod()

# 3. 绘图
fig, ax1 = plt.subplots(figsize=(12,6))

# 左轴：超额净值
ax1.plot(df['Date'], excess_nv_combo,
         label='宏观+情绪微调 超额净值', color='tab:blue', linewidth=1.8)
ax1.axhline(1.0, linestyle='--', linewidth=1, color='gray')
ax1.set_xlabel('日期')
ax1.set_ylabel('超额净值', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True, alpha=0.3)

# 右轴：调后仓位
ax2 = ax1.twinx()
ax2.plot(df['Date'], combo_adj_pos,
         label='调后仓位', color='tab:orange', linewidth=1.4, alpha=0.8)
ax2.set_ylabel('仓位比例', color='tab:orange')
ax2.tick_params(axis='y', labelcolor='tab:orange')

# 合并图例
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
ax1.legend(h1+h2, l1+l2, loc='upper left')

plt.title('宏观+情绪微调策略：超额净值与仓位走势')
plt.tight_layout()
plt.show()


In [ ]:
# === 统计各仓位段表现 ===============================================
# 1. 取仓位序列与超额收益序列
pos_series    = combo_adj_pos.round(2)          # 取两位小数，方便分组
excess_series = combo_opt_ex                  # (combo_adj_ret - bench_ret)

# 2. 遍历，切分连续持仓段
segments = []  # 记录 (仓位, 时长, 段累计超额收益)
start = 0
for i in range(1, len(pos_series)):
    if pos_series.iat[i] != pos_series.iat[start]:
        seg_ex = excess_series.iloc[start:i]
        cum_ex = (1 + seg_ex).prod() - 1
        seg_len = i - start
        segments.append((pos_series.iat[start], seg_len, cum_ex))
        start = i
# 最后一段
seg_ex = excess_series.iloc[start:]
cum_ex = (1 + seg_ex).prod() - 1
segments.append((pos_series.iat[start], len(pos_series)-start, cum_ex))

seg_df = pd.DataFrame(segments, columns=['仓位', '时长', '段超额收益'])

# 3. 汇总统计
stat_df = (
    seg_df
    .groupby('仓位')
    .agg(
        次数      = ('时长', 'size'),
        平均持有天数 = ('时长', 'mean'),
        胜率      = ('段超额收益', lambda x: (x > 0).mean()),
        平均超额收益 = ('段超额收益', 'mean')
    )
    .round(4)
    .reset_index()
    .sort_values('仓位', ascending=False)
)

display(stat_df.style.format({'胜率':'{:.2%}', '平均超额收益':'{:.2%}'}))

In [ ]:
segments=[]
start=0
for i in range(1,len(df)):
    if (df['Signal_macro'].iat[i],df['Signal_sentiment'].iat[i])!=(df['Signal_macro'].iat[start],df['Signal_sentiment'].iat[start]):
        seg_ex=combo_opt_ex.iloc[start:i]
        segments.append((df['Signal_macro'].iat[start],df['Signal_sentiment'].iat[start],i-start,(1+seg_ex).prod()-1))
        start=i
seg_ex=combo_opt_ex.iloc[start:]
segments.append((df['Signal_macro'].iat[start],df['Signal_sentiment'].iat[start],len(df)-start,(1+seg_ex).prod()-1))

stat=(pd.DataFrame(segments,columns=['宏观信号','情绪信号','段长度','段超额收益'])
      .groupby(['宏观信号','情绪信号'])
      .agg(次数=('段长度','size'),
           平均持有天数=('段长度','mean'),
           胜率=('段超额收益',lambda x:(x>0).mean()),
           平均超额收益=('段超额收益','mean'))
      .round(4).reset_index()
      .sort_values(['宏观信号','情绪信号'],ascending=[False,False]))

stat

In [ ]:
segments=[]
start=0
for i in range(1,len(df)):
    if (df['Signal_macro'].iat[i],df['Signal_sentiment'].iat[i])!=(df['Signal_macro'].iat[start],df['Signal_sentiment'].iat[start]):
        seg_ex=combo_adj_ret2.iloc[start:i]
        segments.append((df['Signal_macro'].iat[start],df['Signal_sentiment'].iat[start],i-start,(1+seg_ex).prod()-1))
        start=i
seg_ex=combo_adj_ret2.iloc[start:]
segments.append((df['Signal_macro'].iat[start],df['Signal_sentiment'].iat[start],len(df)-start,(1+seg_ex).prod()-1))

stat=(pd.DataFrame(segments,columns=['宏观信号','情绪信号','段长度','段超额收益'])
      .groupby(['宏观信号','情绪信号'])
      .agg(次数=('段长度','size'),
           平均持有天数=('段长度','mean'),
           胜率=('段超额收益',lambda x:(x>0).mean()),
           平均超额收益=('段超额收益','mean'))
      .round(4).reset_index()
      .sort_values(['宏观信号','情绪信号'],ascending=[False,False]))

stat